Validaciones

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, col, when, count, lit

In [3]:
ruta = 'C:/Users/omarc/OneDrive/Escritorio/Proyectos VSC/OmarCano/Track Analítico UNI/05_Datos/02_Datos_Procesados'

desertados_df = pd.read_csv(f'{ruta}/um_desercion.csv')
cepre_df = pd.read_csv(f'{ruta}/hm_cepre_uni_limpio.csv')
admisiones_df = pd.read_csv(f'{ruta}/hm_admisiones_uni_limpio.csv')
matriculados_df = pd.read_csv(f'{ruta}/hm_matriculados_uni_limpio.csv')

In [ ]:
desertados_df.groupby(['Deserción', 'Detalle']).size().reset_index(name='Conteo')
#desertados_df.loc[grouped['Deserción']==1].groupby(['Deserción', 'Detalle','max_CicloRelativo']).size().reset_index(name='Conteo')
#desertados_df.count()
#matriculados_df.count()
#matriculados_df['IDHASH'].nunique()
matriculados_df

In [29]:
desertores_df = desertados_df[desertados_df['Deserción'] == 1][['IDHASH']]

resultados_df = matriculados_df[matriculados_df['IDHASH'].isin(desertores_df['IDHASH'])]
filtered_df = matriculados_df[matriculados_df['IDHASH'].str.contains('8304997770D7C2F95317154457FE3C3E2838E232B5C09F')]
filtered_df[['TIPO_MATRICULA', 'MODALIDAD', 'METODOLOGIA','CICLO_RELATIVO','CodYear']].sort_values(by='CodYear')
resultados_df[resultados_df['TIPO_MATRICULA'] == 'Cachimbos']
filtered_df

,IDHASH,COLEGIO_DEPA,COLEGIO_PROV,COLEGIO_DIST,ANIO,PERIODO,TIPO_MATRICULA,DOMICILIO_DEPA,DOMICILIO_PROV,DOMICILIO_DIST,...,NACIMIENTO_DEPA,NACIMIENTO_PROV,NACIMIENTO_DIST,SEXO,MODALIDAD,METODOLOGIA,FACULTAD,ESPECIALIDAD,CICLO_RELATIVO,CodYear
474,8304997770D7C2F95317154457FE3C3E2838E232B5C09F...,Lima,Lima Metropolitana,San Luis,2016,1,Cachimbos,Amazonas,Sin Datos,Sin Datos,...,Puno,San Román,Juliaca,Masculino,Ingreso Directo,Presencial,Ingeniería Mecánica,Ingeniería Mecatrónica,1,20161
10575,8304997770D7C2F95317154457FE3C3E2838E232B5C09F...,Lima,Lima Metropolitana,San Luis,2016,2,Regulares,Amazonas,Sin Datos,Sin Datos,...,Puno,San Román,Juliaca,Masculino,Ingreso Directo,Presencial,Ingeniería Mecánica,Ingeniería Mecatrónica,2,20162
20636,8304997770D7C2F95317154457FE3C3E2838E232B5C09F...,Lima,Lima Metropolitana,San Luis,2017,1,Regulares,Amazonas,Sin Datos,Sin Datos,...,Puno,San Román,Juliaca,Masculino,Ingreso Directo,Presencial,Ingeniería Mecánica,Ingeniería Mecatrónica,3,20171
32396,8304997770D7C2F95317154457FE3C3E2838E232B5C09F...,Lima,Lima Metropolitana,San Luis,2017,2,Regulares,Amazonas,Sin Datos,Sin Datos,...,Puno,San Román,Juliaca,Masculino,Ingreso Directo,Presencial,Ingeniería Mecánica,Ingeniería Mecatrónica,4,20172
50713,8304997770D7C2F95317154457FE3C3E2838E232B5C09F...,Lima,Lima Metropolitana,San Luis,2018,1,Regulares,Amazonas,Sin Datos,Sin Datos,...,Puno,San Román,Juliaca,Masculino,Ingreso Directo,Presencial,Ingeniería Mecánica,Ingeniería Mecatrónica,4,20181
61947,8304997770D7C2F95317154457FE3C3E2838E232B5C09F...,Lima,Lima Metropolitana,San Luis,2018,2,Regulares,Amazonas,Sin Datos,Sin Datos,...,Puno,San Román,Juliaca,Masculino,Ingreso Directo,Presencial,Ingeniería Mecánica,Ingeniería Mecatrónica,4,20182
67738,8304997770D7C2F95317154457FE3C3E2838E232B5C09F...,Lima,Lima Metropolitana,San Luis,2019,1,Regulares,Amazonas,Sin Datos,Sin Datos,...,Puno,San Román,Juliaca,Masculino,Ingreso Directo,Presencial,Ingeniería Mecánica,Ingeniería Mecatrónica,5,20191
84467,8304997770D7C2F95317154457FE3C3E2838E232B5C09F...,Lima,Lima Metropolitana,San Luis,2019,2,Regulares,Amazonas,Sin Datos,Sin Datos,...,Puno,San Román,Juliaca,Masculino,Ingreso Directo,Presencial,Ingeniería Mecánica,Ingeniería Mecatrónica,6,20192
96138,8304997770D7C2F95317154457FE3C3E2838E232B5C09F...,Lima,Lima Metropolitana,San Luis,2020,1,Regulares,Amazonas,Sin Datos,Sin Datos,...,Puno,San Román,Juliaca,Masculino,Ingreso Directo,Presencial,Ingeniería Mecánica,Ingeniería Mecatrónica,7,20201
107923,8304997770D7C2F95317154457FE3C3E2838E232B5C09F...,Lima,Lima Metropolitana,San Luis,2020,2,Regulares,Amazonas,Sin Datos,Sin Datos,...,Puno,San Román,Juliaca,Masculino,Ingreso Directo,Presencial,Ingeniería Mecánica,Ingeniería Mecatrónica,7,20202


In [5]:
# Validación 1: Revisar si los estudiantes con deserción (Deserción=1) tienen max_CodYear menor que el último periodo disponible.
matriculados_df['CodYear'] = matriculados_df['ANIO'] * 10 + matriculados_df['PERIODO']
max_global_codyear = matriculados_df['CodYear'].max()

desertados_flag_1 = desertados_df[
    (desertados_df['Deserción'] == 1) & 
    (desertados_df['max_CodYear'] >= max_global_codyear)
]

print("Validación 1: Estudiantes con deserción pero con max_CodYear incorrecto")
desertados_flag_1

Validación 1: Estudiantes con deserción pero con max_CodYear incorrecto


,IDHASH,min_CodYear,max_CodYear,max_CicloRelativo,total_records,Deserción,Detalle


In [6]:
# Validación 2: Verificar si hay estudiantes con detalle "Sigue estudiando", "Retomó estudios" o "Finalizó carrera" pero con Deserción=1.
desertados_flag_2 = desertados_df[
    (desertados_df['Detalle'].isin(['Sigue estudiando', 'Retomó estudios', 'Finalizó carrera'])) & 
    (desertados_df['Deserción'] == 1)
]

print("\nValidación 2: Estudiantes con detalle contradictorio y Deserción=1")
print(desertados_flag_2)


Validación 2: Estudiantes con detalle contradictorio y Deserción=1
Empty DataFrame
Columns: [IDHASH, min_CodYear, max_CodYear, max_CicloRelativo, total_records, Deserción, Detalle]
Index: []


In [7]:
# Validación 3: Validar si los estudiantes con deserción (Deserción=1) tienen "Desertó" en el campo Detalle.
desertados_flag_3 = desertados_df[
    (desertados_df['Deserción'] == 1) & 
    (desertados_df['Detalle'] != 'Desertó')
]

print("\nValidación 3: Estudiantes con deserción pero con un Detalle incorrecto")
print(desertados_flag_3)


Validación 3: Estudiantes con deserción pero con un Detalle incorrecto
Empty DataFrame
Columns: [IDHASH, min_CodYear, max_CodYear, max_CicloRelativo, total_records, Deserción, Detalle]
Index: []


In [8]:
# Validación 4: Asegurarse de que los estudiantes con "Desertó" en Detalle tengan Deserción=1.
desertados_flag_4 = desertados_df[
    (desertados_df['Detalle'] == 'Desertó') & 
    (desertados_df['Deserción'] == 0)
]

print("\nValidación 4: Estudiantes con Detalle 'Desertó' pero con Deserción=0")
print(desertados_flag_4)


Validación 4: Estudiantes con Detalle 'Desertó' pero con Deserción=0
Empty DataFrame
Columns: [IDHASH, min_CodYear, max_CodYear, max_CicloRelativo, total_records, Deserción, Detalle]
Index: []
